In [1]:
import pandas as pd
import numpy as np
import os, os.path
from whoosh import index
from whoosh.index import open_dir
from whoosh.fields import Schema, ID, TEXT, STORED
from whoosh.qparser import QueryParser
from whoosh.qparser import MultifieldParser
from whoosh.query import *
import tarfile
import warnings
warnings.filterwarnings('ignore')
import xml.etree.cElementTree as et
import codecs

In [2]:
# Creates directory: indexdir
if not os.path.exists("indexdir"):
   os.mkdir("indexdir")

In [3]:
# schema =  Schema(url=ID(stored=True), title=TEXT(stored=True), content=TEXT(stored=True))

# ix = index.create_in("indexdir", schema)


In [4]:
# tar = tarfile.open("clinicaltrials_xml.tar.gz", "r:gz")
# members=tar.getmembers()

In [5]:
# for member in members:
#     tar.extractfile(member)

In [6]:
# count=0
# for subdir, dirs, files in os.walk('clinicaltrials.xml'):
#     for file in files:
#         #print os.path.join(subdir, file)
#         filepath = subdir + os.sep + file

#         if filepath.endswith(".xml"):
#             count+=1
# count

In [7]:
# writer = ix.writer()


In [8]:
# schema =  Schema(url=ID(stored=True), title=TEXT(stored=True), content=TEXT(stored=True))
# ix = index.create_in("indexdir", schema)

# import xml.etree.cElementTree as et
# import codecs

# #  This function iterates through cilinicaltrials.xml parent directory to all files in all 
# #  subdirectories. 

# for subdir, dirs, files in os.walk('clinicaltrials.xml/clinicaltrials_xml/002'):
#     for file in files:
#         #print os.path.join(subdir, file)
#         filepath = subdir + os.sep + file
#         parser = et.XMLParser(encoding="utf-8")
#         parsedXML = et.parse(filepath)
#         def getvalueofnode( node ):
#             return node.text if node is not None else None
        
# #         root=parsedXML.getroot()
# #         detailed_description=root.find('detailed_description')
        
        
# #         def get_textblock(nodes):
# #             for n in nodes.iter:
# #                 if n.tag=='textblock':
# #                     return node.text
            
        
        
#         for node in parsedXML.getroot():
#             url = node.attrib.get('url')
#             official_title = node.attrib.get('official_title')
#             detailed_description=node.find('detailed_description/textblock')
#             writer.add_document(url=url, title=official_title, content= getvalueofnode(detailed_description))
# writer.commit()

In [9]:
schema =  Schema(name=ID(stored=True), Title=TEXT(stored=True),content=TEXT(stored=True),gender=TEXT(stored=False),minage=TEXT(stored=False),maxage=TEXT(stored=False))
ix = index.create_in("indexdir", schema)
writer = ix.writer()

In [10]:
#traversing through all the files in the directory
for j in [b for b in os.listdir("clinicaltrials_xml/clinicaltrials_xml/020/") if b.startswith('0')]:
    for k in os.listdir("clinicaltrials_xml/clinicaltrials_xml/020/"+j+"/"):
        root=et.parse("clinicaltrials_xml/clinicaltrials_xml/020/"+j+"/"+k).getroot()
#Got the root element now it is time to get into the textbloc of brief_summary and the detail_description
        try:
            content_text=(root.find('./brief_summary/textblock').text)+(root.find('./detailed_description/textblock').text)
        except:
            if root.find('./detailed_description/textblock') is None and root.find('./detailed_description/textblock') is None:
                content_text=' '
            elif root.find('./brief_summary/textblock') is None:
                content_text=root.find('./detailed_description/textblock').text
            else:
                content_text=root.find('./brief_summary/textblock').text

        try:
            title = (root.find('./brief_title').text)+(root.find('./official_title').text)
        except:
            if root.find('./brief_title') is None and root.find('./official_title') is None:
                title = " "
            elif root.find('./brief_title') is None:
                title = root.find('./official_title').text
            else:
                title = root.find('./brief_title').text

        try:
            url=root.find('./required_header/url').text
        except:
            url=" "
        try:
            if root.find('./eligibility/minimum_age').text=='N/A':
                miage=u'0'
            else:
                miage=root.find('./eligibility/minimum_age').text
        except:
            miage=u'0'
        try:
            if root.find('./eligibility/maximum_age').text=='N/A':
                maage=u'200'
            else:
                maage=root.find('./eligibility/maximum_age').text
        except:
            maage=u'200'
        try:
            ge=root.find('./eligibility/gender').text
        except:
            ge=' '
#Schema(name=ID(stored=True), Title=TEXT(stored=True),content=TEXT(stored=True),gender=TEXT(stored=True),minage=TEXT(stored=True),maxage=TEXT(stored=True))
        writer.add_document(name=url,content=content_text,Title=title,gender=ge,minage=miage,maxage=maage)
writer.commit()

In [11]:
ix = open_dir("indexdir")
qp = MultifieldParser(["content",'Title'], schema =ix.schema)
q = qp.parse(u"sarcoma")

with ix.searcher() as s:
    results = s.search(q, limit=10)
    print(results[0:10])

[<Hit {'Title': 'SARC024: A Blanket Protocol to Study Oral Regorafenib in Patients With Refractory Liposarcoma, Osteogenic Sarcoma, and Ewing SarcomasA Blanket Protocol to Study Oral Regorafenib in Patients With Refractory Liposarcoma, Osteogenic Sarcoma, and Ewing/Ewing-like Sarcomas', 'content': '\n      Although regorafenib was approved for use in patients who had progressive GIST despite\n      imatinib and/or sunitinib on the basis of phase II and phase III data, it has not been\n      examined in a systematic fashion in patients with other forms of sarcoma.\n\n      Given the activity of sorafenib, sunitinib and pazopanib in soft tissue sarcomas, and\n      evidence of activity of sorafenib in osteogenic sarcoma and possibly Ewing/Ewing-like\n      sarcoma, there is precedent to examine SMOKIs (small molecule oral kinase inhibitors) such\n      as regorafenib in sarcomas other than GIST. It is also recognized that SMOKIs (small\n      molecule oral kinase inhibitors)such as regor

In [12]:
cancer='sarcoma'
gene='cdk4'

query_b = MultifieldParser(['Title','content'], ix.schema).parse('{} OR {}'.format(cancer, gene))
with ix.searcher() as srch:
    res_b = srch.search(query_b, limit=20)
    results=[(i['Title'],i['name'],i['content']) for i in res_b]
print(results[0])

('SARC024: A Blanket Protocol to Study Oral Regorafenib in Patients With Refractory Liposarcoma, Osteogenic Sarcoma, and Ewing SarcomasA Blanket Protocol to Study Oral Regorafenib in Patients With Refractory Liposarcoma, Osteogenic Sarcoma, and Ewing/Ewing-like Sarcomas', 'https://clinicaltrials.gov/show/NCT02048371', '\n      Although regorafenib was approved for use in patients who had progressive GIST despite\n      imatinib and/or sunitinib on the basis of phase II and phase III data, it has not been\n      examined in a systematic fashion in patients with other forms of sarcoma.\n\n      Given the activity of sorafenib, sunitinib and pazopanib in soft tissue sarcomas, and\n      evidence of activity of sorafenib in osteogenic sarcoma and possibly Ewing/Ewing-like\n      sarcoma, there is precedent to examine SMOKIs (small molecule oral kinase inhibitors) such\n      as regorafenib in sarcomas other than GIST. It is also recognized that SMOKIs (small\n      molecule oral kinase inh

In [7]:
filepath='clinicaltrials.xml/clinicaltrials_xml/020/02000/NCT02000011.xml'

parser = et.XMLParser(encoding="utf-8")
parsedXML = et.parse(filepath, parser=parser)
for node in parsedXML.getroot():
    d=node.findall('detailed_description')
    for text in d:
        t=text.find('textblock')
        print(t.text)

In [8]:
with open('clinicaltrials.xml/clinicaltrials_xml/020/02000/NCT02000011.xml', 'r') as f:
    h=f.read()
    f.close()

    print(h)

<?xml version="1.0" encoding="UTF-8"?>
<clinical_study rank="84899">
  <!-- This xml conforms to an XML Schema at:
    https://clinicaltrials.gov/ct2/html/images/info/public.xsd -->
  <required_header>
    <download_date>ClinicalTrials.gov processed this data on April 10, 2017</download_date>
    <link_text>Link to the current ClinicalTrials.gov record.</link_text>
    <url>https://clinicaltrials.gov/show/NCT02000011</url>
  </required_header>
  <id_info>
    <org_study_id>2013-A01038-37</org_study_id>
    <secondary_id>2013-33</secondary_id>
    <nct_id>NCT02000011</nct_id>
  </id_info>
  <brief_title>Interest of a Geriatric Intervention Plan Associated to a Comprehensive Geriatric Assessment on Autonomy, Quality of Life and Survival of Patients Aged 70 Years Old and More Surgically Treated for a Resectable Cancer (Thoracic, Digestive or Urologic). Randomized Multicentric Study</brief_title>
  <acronym>epigac</acronym>
  <official_title>Interest of a Geriatric Intervention Plan Associ

In [233]:
%history

import pandas as pd
import numpy as np
import os, os.path
from whoosh import index
from whoosh.index import open_dir
from whoosh.fields import Schema, ID, TEXT, STORED
from whoosh.qparser import QueryParser
from whoosh.query import *
import tarfile
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os, os.path
from whoosh import index
from whoosh.index import open_dir
from whoosh.fields import Schema, ID, TEXT, STORED
from whoosh.qparser import QueryParser
from whoosh.query import *
import tarfile
import warnings
warnings.filterwarnings('ignore')
if not os.path.exists("indexdir"):
   os.mkdir("indexdir")
schema =  Schema(url=ID(stored=True), title=STORED, content=TEXT)
ix = index.create_in("indexdir", schema)
writer = ix.writer()
# tar = tarfile.open("clinicaltrials_xml.tar.gz", "r:gz")
# members=tar.getmembers()
# for member in members:
#     tar.extractfile(member)
count=0
for subdir, dirs, files in os.walk('clinicaltrials.xml'):
    for f